In [75]:
import pandas as pd 
import numpy as np

In [76]:
df=pd.read_csv("adult.csv",encoding='latin-1')
df[df == '?'] = np.nan
for col in ['workclass', 'occupation', 'native.country']:
    df[col].fillna(df[col].mode()[0], inplace=True)
X = df.drop(['income'], axis=1)
y = df['income']

In [77]:
leng=X.shape[0]

In [78]:
X_shap,y_shap=X[:int(leng*0.7)].copy(),y[:int(leng*0.7)].copy()
X_background,y_background=X[int(leng*0.7):].copy(),y[int(leng*0.7):].copy()

In [79]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from numpy import int64
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
import pickle

In [80]:
categorical = ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'race', 'sex', 'native.country']
for feature in categorical:
        le = preprocessing.LabelEncoder()
        X_shap[feature] = le.fit_transform(X_shap[feature])
        X_background[feature] = le.transform(X_background[feature])
scaler = StandardScaler()

X_shap = pd.DataFrame(scaler.fit_transform(X_shap), columns = X.columns)

X_background = pd.DataFrame(scaler.transform(X_background), columns = X.columns)

y_shap[y_shap=="<=50K"]=0
y_shap[y_shap==">50K"]=1
y_background[y_background=="<=50K"]=0
y_background[y_background==">50K"]=1
y_shap=y_shap.astype(int64)
y_background=y_background.astype(int64)

In [81]:
import shap
import matplotlib.pyplot as plt
import os
import time
shap_values=None
def load_pre(filename,isTree=True,save=True):
    loaded_model = pickle.load(open(filename, 'rb'))
    # y_pred=loaded_model.predict(X_shap)
    # print(accuracy_score(y_shap,y_pred))
    global shap_values

    if isTree:
        explainer = shap.TreeExplainer(loaded_model)
        shap_values = explainer.shap_values(X=X_shap)
    else:
        explainer = shap.Explainer(loaded_model,X_background)
        shap_values = explainer.shap_values(X_shap)
    # print(shap_values)
    if isinstance(shap_values,list):
        # shap.summary_plot(shap_values[0],X_shap,max_display=10)
        # shap.summary_plot(shap_values[1],X_shap,max_display=10)
        if save:
            np.save("/home/zyxing/adult_income/shaps/"+os.path.basename(filename)+".npy",np.array(shap_values[1]))
        else:
            shap.summary_plot(shap_values[1],X_shap,max_display=5,show=False)
            plt.savefig(str(time.time())+".png")
    else:
        if save:
            np.save("/home/zyxing/adult_income/shaps/"+os.path.basename(filename)+".npy",np.array(shap_values))
        else:
            shap.summary_plot(shap_values,X_shap,max_display=5,show=False)
            plt.savefig(str(time.time())+".png")

    

In [82]:
# import os
# os.path.basename("/home/zyxing/adult_income/models/bst0.sav")

In [83]:
def findAllFile(base):
    for root, ds, fs in os.walk(base):
        for f in fs:
            fullname = os.path.join(root, f)
            yield fullname

In [84]:
# for i in findAllFile("/home/zyxing/adult_income/models"):
#     print(i)

In [85]:
# shap_values

In [86]:
# load_pre("/home/zyxing/adult_income/models/bst0.sav")
# load_pre("/home/zyxing/adult_income/models/clf0.ptl",save=False)
# load_pre("/home/zyxing/adult_income/models/clf16.ptl",save=False)
# load_pre("/home/zyxing/adult_income/models/logreg0.sav",isTree=False)
# load_pre("/home/zyxing/adult_income/models/rf0.sav")

In [87]:
type(shap_values)

NoneType

In [88]:
count1=0

for i in findAllFile("/home/zyxing/adult_income/models"):
    name=os.path.basename(i)
    # if name.startswith("rf"):
    #     load_pre(i)
    if name.startswith("logreg"):
        load_pre(i,False)
#     elif name.startswith("bst"):
#         load_pre(i)
#     elif name.startswith("clf"):
#         load_pre(i)
    count1+=1
    print(count1)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
